# `CrystalMap` API

* Quick and dirty description of the (more or less) full `CrystalMap` API so far.
* Will of course remove lots of stuff here and make it user friendly before merging into orix-demos.
* Notable things missing:
    * `__setitem__`
* Many try/except and input checks are implemented. Try to break things. An explanatory error message *should* be raised.

<!--
# Introduction

This notebook illustrates clustering of Ti crystal orientations using data obtained from a highly deformed specimen, using EBSD.

This functionaility has been checked to run in orix-0.2.0 (December 2019). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/orix-demos/issues
-->

# Contents

1. <a href='#load'> Temporary load functions</a>
2. <a href='#init'> Initialize a CrystalMap object</a>
3. <a href='#crystalmap'> CrystalMap</a>
4. <a href='#phaselist'> PhaseList</a>
5. <a href='#phase'> Phase</a>
6. <a href='#examples'> Examples</a>

Import orix classes and various dependencies

In [ ]:
%matplotlib qt5

# Important external dependencies
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import h5py  # Needed for h5ebsd reader

# orix dependencies (not tested)
from orix.crystal_map import CrystalMap, PhaseList, Phase
from orix.quaternion.rotation import Rotation

# <a id='load'></a> 1. Temporary load functions

### .ang file created by EMsoft's EMdpmerge program

Not at all done, just something quick to read all necessary bits.

In [ ]:
def load_ang(filename):
    data = np.loadtxt(filename)
    rows, cols = data.shape

    euler = np.radians(data[:, :3])
    x = data[:, 3]
    y = data[:, 4]
    iq = data[:, 5]
    ci = data[:, 6]

    fit = np.ones_like(iq)
    if cols > 8:
        fit = data[:, 7]
        phase = data[:, 8]
    else:
        phase = data[:, 7]

    properties = {
        'iq': iq,
        'ci': ci,
        'fit': fit,
    }

    # Read header
    header = []
    for line in open(filename):
        l = line.strip()
        if l.startswith("#"):
            header.append(l.rstrip())

    def get_phase_from_ang(header):
        phases = {}
        for i, l in enumerate(header):
            if l.startswith("# Phase"):
                phase_id = re.search("# Phase( +)([0-9]+)", l).group(2)
                material_name = ''
                formula = ''
                try:
                    material_name = re.search(
                        "# MaterialName( +)([A-z]+)/([A-z]+)", header[i + 1],
                    ).group(2)
                    formula= re.search(
                        "# Formula( +)([A-z]+)/([A-z]+)", header[i + 2]).group(
                        2)
                except (AttributeError, IndexError):
                    material_name = re.search(
                        "# MaterialName( +)([A-z]+)", header[i + 1],
                    ).group(2)
                    formula= re.search(
                        "# Formula( +)([A-z]+)", header[i + 2]).group(2)
                phases[phase_id] = {
                    'material_name': material_name,
                    'formula': formula,
                    'symmetry': re.search(
                        "# Symmetry( +)([A-z0-9]+)", header[i + 4],
                    ).group(2),
                }
        return phases

    # Get phases
    phases = get_phase_from_ang(header)

    # Get symmetries and minerals
    symmetry = []
    mineral = []
    for p in phases.values():
        symmetry.append(p['symmetry'])
        mineral.append(p['material_name'])

    return x, y, phase, euler, properties, symmetry, mineral

### .h5 file created by EMsoft's EMEBSDDI program

Not at all done, just something quick to read all necessary bits.

In [ ]:
def load_h5ebsd(filename, refined=False, **kwargs):
    mode = kwargs.pop("mode", "r")
    f = h5py.File(filename, mode=mode, **kwargs)

    scan_group = f["Scan 1"]
    ebsd_group = scan_group["EBSD"]
    data_group = ebsd_group["Data"]
    header_group = ebsd_group["Header"]

    # Get scan coordinates
    nx = header_group["nColumns"][()]
    ny = header_group["nRows"][()]
    dx = header_group["Step X"][()]
    dy = header_group["Step Y"][()]
    x = np.tile(np.arange(nx*dx, step=dx), reps=ny)
    y = np.tile(np.arange(ny*dy, step=dy), reps=nx)
    z = np.zeros_like(x)

    # Get phase
    phase = data_group["Phase"][()]

    # Get crystal symmetry
    point_group = re.search(
        r"\[([A-Za-z0-9_]+)\]",
        ebsd_group["Header/Phase/1/Point Group"][()][0].decode()
    ).group(1)

    # Get orientations
    if refined:
        orientation_dataset_name = "RefinedEulerAngles"
    else:
        orientation_dataset_name = "EulerAngles"
    euler = data_group[orientation_dataset_name][()]

    # Get properties
    properties = {}
    quality_metric_names = [
        "AvDotProductMap",
        "CI",
        "IQ",
        "ISM",
        "OSM",
    ]
    for metric_name in quality_metric_names:
        metric = data_group[metric_name][()]
        if metric.ndim > 1:
            metric = metric.ravel()
        properties[metric_name] = metric

    return x, y, z, phase, euler, properties, point_group

### Utility program to get map shape and step size from x, y arrays

All this will be done internally...

In [ ]:
def get_shape_and_step_sizes(x, y=None, z=None):
    shape = []
    step_sizes = []
    for i, direction in enumerate([z, y, x]):
        if direction is not None:
            unique_sorted = np.sort(np.unique(direction))
            step = unique_sorted[1] - unique_sorted[0]
            length = int((direction.max() + step) / step)
            shape.append(length)
            step_sizes.append(step)
    return tuple(shape), step_sizes

# <a id='init'></a> 2. Initialize a CrystalMap object

Set input file and file path.

.ang and .h5ebsd files are temporarily available here: http://folk.ntnu.no/hakonwii/files/orix-demos/

In [ ]:
datadir = '/home/hakon/phd/data/jarle_emsoft/sdss/emsoft'
#fname = 'sdss_austenite_dp.h5'
fname = 'sdss_ferrite_austenite.ang'
file = os.path.join(datadir, fname)

In [ ]:
#x, y, z, phase, euler, props, symmetry = load_h5ebsd(file)

In [ ]:
x, y, phase, euler, props, symmetry, mineral = load_ang(file)

Reshape 1D arrays to map shape (will be done internally when current readers are expanded)

In [ ]:
shape, step_sizes = get_shape_and_step_sizes(x, y)

rotations = Rotation.from_euler(euler.reshape(shape + (3,)))

phase = phase.reshape(shape)

prop = {}
for k, v in props.items():
    prop[k] = v.reshape(shape)

Print `__init__` docstring

In [ ]:
print(CrystalMap.__init__.__doc__)    

Create crystal map

In [ ]:
cm = CrystalMap(
    rotations=rotations,
    phase_id_map=phase,
    phase_name=mineral,
    symmetry=symmetry,
    prop=prop,
    step_sizes=step_sizes,
)

# <a id='crystalmap'></a> 3. CrystalMap

Print class description

In [ ]:
print(CrystalMap.__doc__)

`__repr__` (inspired by MTEX)

In [ ]:
cm

Custom, private attributes

In [ ]:
[i for i in dir(cm) if i.startswith('_') and not i.endswith('__')]

Public attributes and methods

In [ ]:
[i for i in dir(cm) if not i.startswith('_')]

Docstrings of public attributes and their values in the current CrystalMap instance

In [ ]:
print(CrystalMap.all_indexed.__doc__)
cm.all_indexed

In [ ]:
print(CrystalMap.step_sizes.__doc__)
cm.step_sizes

In [ ]:
print(CrystalMap.indexed.__doc__)
cm.indexed

In [ ]:
print(CrystalMap.ndim.__doc__)
cm.ndim

In [ ]:
print(CrystalMap.orientations.__doc__)
cm['austenite'].orientations

In [ ]:
print(CrystalMap.phase_id.__doc__)
cm.phase_id

In [ ]:
# Set during __init__, not a derived attribute, hence no docstring available
cm.phases  # similar CrystalMap's __repr__

In [ ]:
print(CrystalMap.phases_in_map.__doc__)
cm.phases_in_map

In [ ]:
print(CrystalMap.prop.__doc__)
cm.prop

In [ ]:
print(CrystalMap.rotations.__doc__)
cm.rotations

In [ ]:
print(CrystalMap.scan_unit.__doc__)

print(cm.scan_unit)

cm.scan_unit = 'um'
print(cm.scan_unit)

In [ ]:
print(CrystalMap.shape.__doc__)
cm.shape

In [ ]:
print(CrystalMap.size.__doc__)
cm.size

Docstrings of CrystalMap methods

In [ ]:
print(CrystalMap.plot_phase.__doc__)

In [ ]:
print(CrystalMap.plot_prop.__doc__)

Indexing/slicing ...

... by map position (slices)

In [ ]:
(y0, y1) = (20, 40)
(x0, x1) = (50, 60)
cm1 = cm[y0:y1, x0:x1]
cm1

In [ ]:
cm1.size

... by phase name

In [ ]:
cm2 = cm['austenite']
cm2

In [ ]:
# np.sum(cm2.indexed) / cm.size
cm2.size / cm.size

In [ ]:
cm3 = cm['austenite', 'ferrite']
cm3

... by (chained) conditional(s)

In [ ]:
cm4 = cm[cm.phase_id == 1]
cm4

In [ ]:
cm5 = cm[cm.ci > 0.81]
cm5

In [ ]:
cm6 = cm[(cm.iq > np.mean(cm.iq)) & (cm.phase_id == 1)]
cm6

Add property

In [ ]:
cm.prop['ci_times_iq'] = cm.ci * cm.iq
print(cm)
print("\n", cm.ci_times_iq)

# <a id='phaselist'></a> 4. PhaseList

In [ ]:
print(PhaseList.__doc__)

Get from CrystalMap

In [ ]:
phases = cm.phases  # As shown above
phases

Custom, private attributes

In [ ]:
[i for i in dir(phases) if i.startswith('_') and not i.endswith('__')]

Public attributes

In [ ]:
[i for i in dir(phases) if not i.startswith('_')]

Inspect properties

In [ ]:
print(PhaseList.size.__doc__)
phases.size

In [ ]:
print(PhaseList.phase_ids.__doc__)
phases.phase_ids

In [ ]:
print(PhaseList.names.__doc__)
phases.names

In [ ]:
print(PhaseList.symmetries.__doc__)
phases.symmetries

In [ ]:
# Should perhaps add this property
#phases.symmetry_names

In [ ]:
print(PhaseList.colors.__doc__)
phases.colors

In [ ]:
print(PhaseList.colors_rgb.__doc__)
phases.colors_rgb

Initialize a PhaseList...

In [ ]:
print(PhaseList.__init__.__doc__)

... from input

In [ ]:
PhaseList(
    names=['al', 'cu'],
    symmetries=['m-3m', 'm3m'],  # Note that m3m = m-3m
    colors=['lime', 'xkcd:violet'],
    phase_ids=[0, 1],
)

... from a list of Phase objects

In [ ]:
al = Phase(name='al', symmetry='m-3m')
cu = Phase(name='cu')

#PhaseList([al, cu])  # Not working, will implement
PhaseList(al)

Index PhaseList...

In [ ]:
print(PhaseList.__getitem__.__doc__)
# This doc should perhaps be moved to a more accessible place

... by phase name

In [ ]:
phases['austenite']

In [ ]:
phases['austenite', 'ferrite']

... by phase id

In [ ]:
phases[1]

In [ ]:
phases[1, 2]

... by slice

In [ ]:
phases[1:]

Add phase to PhaseList

In [ ]:
phases2 = phases.deepcopy()
phases2['sigma'] = '4/mmm'
phases2

# <a id='phase'></a> 5. Phase

In [ ]:
print(Phase.__doc__)

No custom, private attributes.

Public attributes

In [ ]:
[i for i in dir(Phase) if not i.startswith('_')]

In [ ]:
print(Phase.color.__doc__)


From PhaseList by...

... phase name

In [ ]:
phases['austenite']

... phase ID

In [ ]:
phases[1]

Initialize

In [ ]:
Phase('au', symmetry='m-3m', color='salmon')

# <a id='examples'></a> 6. Examples

Plot phase map with property as alpha

In [ ]:
cm.plot_phase('ci');

In [ ]:
# Alternatively, return data array plotted, figure, image and axes
#data, fig, im, ax = cm.plot_phase(
#    overlay='ci',
#    scalebar=False,
#    legend=False,
#    padding=False,
#)

# And e.g. save it as map of pixels or use it as navigatior in HyperSpy
#plt.imsave(os.path.join(datadir, 'phase_ci.png'), arr=data)

In [ ]:
# Per phase
cm['austenite'].plot_phase('ci');

Plot property

In [ ]:
cm.plot_prop('iq');

In [ ]:
#data, fig, im, ax = cm.plot_prop(
#    prop='iq',
#    scalebar=False,
#    colorbar=False,
#    padding=False,
#    cmap='viridis',
#)

In [ ]:
# Per phase
cm['ferrite'].plot_prop('iq', cmap='viridis');

In [ ]:
# 2D slicing
cm[20:40, 50: 100].plot_prop('ci');

In [ ]:
# Conditional slicing
cm[cm.ci > 0.81].plot_prop('ci', cmap='winter');

# Chained conditional slicing
#cm[(cm.ci > 0.81) & (cm.phase_id == 1)].plot_prop('ci', cmap='winter');

Plot histogram of a property per phase

In [ ]:
# Change colors (for fun)
cm.phases['ferrite'].color = (1, 0, 0)  # or 'r' or 'red'
cm.phases['austenite'].color = (0, 1, 0)  # or 'lime'

# Property of interest
this_prop = 'ci'

# Plot phase map again to see color changes
cm.plot_phase(this_prop)

# Declare lists for plotting
data = []
labels = []
colors = []

# Get property values, name and color per phase
for _, p in cm.phases_in_map:
    labels.append(p.name)
    colors.append(p.color)

    mask = cm[p.name].indexed
    data.append(cm[p.name].prop[this_prop][mask])
    # Alternatively, without the two above lines:
    #data.append(cm[p.name].ci.compressed())
    # Need compressed() because cm['some_masking'].ci returns a
    # masked array, which does not play nicely with matplotlib.hist.
    # This works because properties are made available as properties via
    # __getattr__

# Nice bar plot with property histogram per phase
fig, ax = plt.subplots()
ax.hist(
    data,
    bins=20,
    histtype='bar',
    density=True,
    label=labels,
    color=colors
)
ax.set_xlabel(this_prop)
ax.set_ylabel("Frequency")
ax.legend();